# 深度学习计算

## 层和块
    
神经网络模型（或者单个层）：输入、输出与参数。输入在网络层中向前传播，误差经过梯度计算向后传播（BP算法）。深度学习广泛应用于cv、nlp等方向，和强化学习结合形成深度强化学习，在金融量化中也有着广泛的应用。在现在的深度学习任务中，神经网络的模型往往是很复杂的，层数会达到数百层。因此，研究网络的部分（组件）比研究单个层或者整个模型本身就显得更加重要。神经网络块相当于编程的一个类，它的任一子类都需要定义将输入转换为输出的向前传播函数，并且能存储必要的参数（块也可以没有参数）。由于有自动微分，反向传播函数与梯度计算由系统自动完成。神经网络块具有多功能性，可以描述单个的层、由多个层构成的组件，甚至整个模型本身。使用块的好处就是可以将块递归地拼接成一个更大的组件。

RK:我们之前所说的层相当于网络模型的组成层，由多个神经元构成。比如隐藏层。现在说的层一般指模型中的操作层，比如线性层。当然，可以用线性层定义一个隐藏层。

- torch.nn: pytorch自带的一个函数库，包含了神经网络中使用的一些常用的模块，比如nn.Linear、nn.Conv2d等。

- nn.Module: 所有神经网络模块的基类，所有定义的模块都要继承它，常用于高级的功能模块、网络层的搭建。上述层的定义也继承了该基类。

- nn.functional: 也是nn中常用的一个模块，nn中大多数层在functional中都有一个函数与之对应，比如F.linear、F.conv2d等。nn.Module和 
    nn.functional在性能上没有太大差异，也能实现相同的功能。区别在于:nn中的层由类定义，继承nn.Module，可以自动提取可学习的参数；而   
    nn.functional中的由函数定义。因此模块如果有可学习的参数，一般用nn.Module，并且放在构造函数__init__()中，比如线性层、卷积层；如 
    果模型不具有可学习参数，则可以放在__init__()中也可以不放，当放在前向传播函数中时就用相应的函数代替，比如激活层、池化层。
    
- nn.Sequential: 继承自nn.Module，相当于一个可以包装层的有序容器，模块将按照构造函数中的传递顺序依次被添加到这个容器中执行

下段代码通过实例化nn.Sequential类构建了一个全连接的神经网络，隐藏层与输出层都是nn.Linear类的实例化，并且层作为Sequential的参数，执行顺序是按照其参数传递的顺序进行。

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X=torch.rand(2,20)
net(X)
print(X.grad)

None


### 5.1.1 自定义块

我们在自定义块需要知道：
    
- 需要继承nn.Module类（利用super()），并定义向前传播函数；

- 层是否要放在构造函数中；
    
- 只要定义了前向传播函数，通过自动微分，系统自动实现反向传播函数。

In [4]:
class MLP(nn.Module):
    
    def __init__(self):
        #调用父类的构造函数进行必要的初始化
        super().__init__()
        #定义两个全连接层，都是nn.Linear的实例化，并被赋予默认值
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
        
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))
    
net=MLP()
net(X)

tensor([[-0.0023, -0.1800, -0.2153, -0.1431,  0.1024,  0.1512,  0.1314,  0.0312,
          0.3083, -0.0305],
        [ 0.0042, -0.1614, -0.0606, -0.0706,  0.1020,  0.2080,  0.0892, -0.2596,
          0.2909, -0.0515]], grad_fn=<AddmmBackward0>)

F.relu()与nn.ReLU()的区别：F.relu()是函数调用，一般用在forward函数中；nn.ReLU()是模块调用，一般用于定义网络的层。
块的基本功能有：

- 将输入数据作为其向前传播函数的参数；

- 通过向前传播函数来生成输出；

- 计算输出关于输入的梯度，可通过其反向传播函数进行访问；

- 存储和访问向前传播计算所需的参数；

- 根据需要初始化参数。

### 5.1.2 顺序块

为了构建一个顺序块，需要定义两个关键函数：
    
- 一种将块逐个加入到列表中的函数。

- 一种前向传播函数，用于将输入按照追加块的顺序进行传递。

下段代码构建了一个顺序块MySequential，功能与Sequential类相同。

In [3]:
#当不知道传递多少个实参的时候用*
def animal(first_ani,*args):    
    print("The first animal's name is",first_ani)
    print(*args)
        
animal('cat','dog','pig')#调用函数

The first animal's name is cat
dog pig


In [4]:
#enumerate的用法
s=[1,2,3,4]
#此处遍历对象是列表，也可以是字符串、字典等
e=enumerate(s)
#返回的是一个enumerate对象，包含索引与值
print(e)
for index,value in e:
    print((index,value))#访问索引以及其对应的值

(0, 1)
(1, 2)
(2, 3)
(3, 4)


In [7]:
class MySequential(nn.Module):
    #因为不知道传递多少数据那么用*
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            self._modules[str(idx)]=module#有序字典
            #idx被转换成字符串作为关键字，与module值构成键值对，依次存放于有序字典self_modules中。
    #self._modules是有序字典
    def forward(self,X):
        #有序字典保证了按照存放顺序进行遍历
        #访问有序字典的值，有序字典访问for index,values in 有序字典,如果只是访问值就用.values
        for block in self._modules.values():
            X=block(X)
        return X

net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.1812, -0.1662, -0.1065,  0.1727, -0.2045,  0.1291,  0.0788,  0.1017,
          0.3599, -0.0027],
        [ 0.1275, -0.0719, -0.0779,  0.1029, -0.1994,  0.0236,  0.0592,  0.0708,
          0.3223,  0.0426]], grad_fn=<AddmmBackward0>)

### 在前向传播函数中执行代码

Sequential类将模块简单地串联起来，使得模型构造变得简单。当需要更强的灵活性时，如想要执行python控制流、执行任意的数学运算等，已有的层不能完成这些任务，我们就还需要定义其他的块。

有时候，我们会想让一些参数的值保持不变，而不是想去更新它，我们将这类参数称之为常数参数。下段代码实现了一个神经网络模型，不过与之前不同的是，我们添加了一个计算函数f(x)=w·x的层，并在前向传播函数中添加了一个控制流。

In [6]:
class FixedHiddenMLP(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20),requires_grad=False)#不更新参数，从而是参数保持不变
        #self.rand_weight就是常数参数，保持不变
        self.linear=nn.Linear(20,20)
        
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight))
        #复用全连接层，参数共享
        X=self.linear(X)
        #控制流，只是给了告诉我们在定义模块的时候可以添加控制流
        while X.abs().sum()>1:
            X/=2
        return X.sum()
    
net=FixedHiddenMLP()
net(X)

tensor(0.1087, grad_fn=<SumBackward0>)

我们可以混合搭配各种组合块的方法。下段代码实现了块的嵌套。

In [7]:
class NestMLP(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                               nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
    
    def forward(self,X):
        return self.linear(self.net(X))

chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.1511, grad_fn=<SumBackward0>)

## 5.2 参数管理

在选择了网络模型架构并设置必要的超参数后，假设空间就确定了。此时根据损失函数最小化的策略对网络进行训练，最终得到相应的参数，网络模型也就确定下来，然后对未知数据进行预测。当然，我们就需要将训练好的参数保存下来，以便在其他环境中复用它们。

在前几节，我们只依靠深度学习框架来完成训练工作，对参数没有进行其他具体的操作。为此，接下来的内容，我们将介绍：

- 访问参数，用于调试、诊断、可视化等；

- 参数初始化；

- 在不同模型组件间实现参数共享。

我们从具有单隐藏层的多层感知机开始。

In [2]:
import torch
from torch import nn
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[0.4710],
        [0.2716]], grad_fn=<AddmmBackward0>)

### 5.2.1 参数访问

首先，我们从已有的模型中访问参数。当实例化Sequential类定义模型时，我们可以将模型的层看作一个列表，通过索引的方式访问任何一个层，每层的参数都在其属性（可通过实例来访问的变量）中。我们可以来访问输出层的参数。其中，参数都存储为单精度浮点型，并且参数名称允许唯一标识每个参数。

In [3]:
print(net[2].state_dict())#.state_dict

OrderedDict([('weight', tensor([[ 0.2455, -0.0415,  0.3363, -0.1995,  0.2323, -0.2685,  0.0360,  0.3345]])), ('bias', tensor([0.1505]))])


**目标参数**

访问参数返回的是一个参数类，有参数值、梯度和额外信息。下段代码提取了输出层的偏置,并进一步访问了偏置的具体值。

In [4]:
print(type(net[2].bias))  #返回偏置参数的类型
print(net[2].bias)#可以更新参数
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1505], requires_grad=True)
tensor([0.1505])


In [5]:
net[2].weight.grad==None  #由于还没有调用反向传播，参数的梯度处于初始状态

True

**一次性访问所有参数**

In [13]:
#以元组形式输出每个层的参数，比较方便，如果单独输出用下面的方法就会比较麻烦
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
#print(*[(name,param.shape) for name,param in net[0].state_dict()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([256, 20])) ('bias', torch.Size([256]))


ValueError: too many values to unpack (expected 2)

In [16]:
ft=[]
#net[0].named_parameters()表示提取网络参数的名字和对应的值
for name,param in net[0].named_parameters():
    ft.append((name,param.shape))
print(ft)

ValueError: too many values to unpack (expected 2)

In [18]:

print(type(net[0].named_parameters()))#是个迭代器所以可以用for循环访问索引以及对应的值
print(type(net[0].state_dict()))#是个有序字典不能用for循环进行访问
net[0].state_dict()

<class 'generator'>
<class 'collections.OrderedDict'>


OrderedDict([('weight',
              tensor([[ 0.1690, -0.1930,  0.0068,  ..., -0.1027, -0.1727, -0.0273],
                      [ 0.1415, -0.0509, -0.1632,  ...,  0.0715, -0.0977, -0.0544],
                      [-0.0978,  0.0800, -0.0905,  ...,  0.0530,  0.0345,  0.1542],
                      ...,
                      [-0.0591,  0.1158,  0.1073,  ..., -0.1088,  0.0101,  0.0883],
                      [ 0.2225,  0.1110,  0.0483,  ..., -0.1018,  0.2156, -0.1966],
                      [ 0.0186,  0.0158,  0.1021,  ...,  0.1299,  0.2000, -0.1046]])),
             ('bias',
              tensor([ 0.0118,  0.0827, -0.1007,  0.1862, -0.1996,  0.0961,  0.1203,  0.1449,
                      -0.0968, -0.0993, -0.1682, -0.2071,  0.1839,  0.1995, -0.1628,  0.1873,
                      -0.0887, -0.0270,  0.0512,  0.1104,  0.0323, -0.1739, -0.2141, -0.2207,
                       0.1328, -0.0419, -0.1421, -0.0786,  0.0513,  0.1232,  0.2155,  0.2114,
                      -0.0615,  0.1347, -0.1

**从嵌套块收集参数**
    
下段代码定义了一个生成块的函数，并将块嵌套成更大的块，其中的层是分层嵌套的，可以通过嵌套索引的方式访问参数。

In [2]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                         nn.Linear(8,4),nn.ReLU())

def block2():
    net=nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())#在字符串中使用i变量的值
    return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

NameError: name 'nn' is not defined

In [3]:
print(rgnet)

NameError: name 'rgnet' is not defined

In [4]:
rgnet[0][1][0].bias.data

NameError: name 'rgnet' is not defined

### 5.2.2 参数初始化
    
之前我们都是默认参数由系统随机初始化，当然我们也可以自定义初始化方法。默认情况下，pytorch会根据一个范围均匀地初始化权重与偏置矩阵，这个范围是根据输入与输出的维度决定。我们可以用nn.init模块实现多种预置初始化方法。
    
**内置初始化**

下段代码将所有权重初始化为标准差为0.01的高斯随机变量，偏置设为0.

In [1]:
def init_constant(m):
    #m的类型是nn.Linear
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)#net间接的调用init_constant函数
net[0].weight.data[0],net[0].bias.data[0]

NameError: name 'net' is not defined

下段代码将参数初始化为给定的参数。

In [35]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

下段代码使用Xavier初始化方法初始化参数。

In [26]:
def init_xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)#用均匀分布填充张量
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)#常数的值为42
        
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.0101, 0.2327, 0.4561, 0.2999])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


**自定义初始化**

我们用任意的分布为权重参数初始化，深度学习框架是没法提供这样的方法。

In [5]:
def my_init(m):
    if type(m)==nn.Linear:
        print('Init',*[(name,param.shape)
                      for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        #如果参数的绝对值大于等于5，则参数保持不变；若绝对值小于5，则将参数置为零
        m.weight.data*=m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

NameError: name 'net' is not defined

In [ ]:
当然，我们可以直接设置参数。

In [37]:
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000, 12.9929, 12.9995, 13.0015])

### 5.2.3 参数绑定

我们可以先定义一个层，然后使用它的参数来设置另一个层的参数，以此来实现参数共享。

In [41]:
shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                  shared,nn.ReLU(),
                  shared,nn.ReLU(),
                  nn.Linear(8,1))
net(X)
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=100
#确保它们是同一个对象，而不是只有相同的值
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
